In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier 

In [ ]:
data = pd.read_csv("../input/csgo-round-winner-classification/csgo_round_snapshots.csv")
pd.set_option('display.max_columns', None)

In [ ]:
data

# Exploratory Data Analysis

In [ ]:
# Winners in different maps

plt.figure(figsize=(10,8))
ax = sns.countplot(x="map", hue="round_winner", data=data)
ax.set(title='Round winners for various maps')
plt.show()

In [ ]:
# Plot for total wins per team

plt.figure(figsize=(10,8))
ax = sns.barplot(x=data['round_winner'].unique(), y=data['round_winner'].value_counts())
ax.set(title='Total number of wins per side', xlabel='Side', ylabel='Wins')
plt.show()

In [ ]:
def preprocess_inputs(df):
    df = df.copy()
    
    # Dropping single valued columns
    
    for column in data.columns:
        if len(data[column].unique())==1:
            df = df.drop([column], axis=1)
    
    # Encoding binary columns
    
    df["bomb_planted"] = df["bomb_planted"].astype(np.int16)
    df["round_winner"] = df["round_winner"].replace({"T":0, "CT":1})
    
    # Encoding the map column using Label Encoder
    
    encoder = LabelEncoder()
    df["map"] = encoder.fit_transform(df["map"]) 
    
    # Splitting the data
    
    y = df["round_winner"]
    X = df.drop("round_winner", axis=1)
    
    # Scaling the data
    
    scaler = RobustScaler()
    X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
    
    # Train-test split
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.8, random_state=42)
        
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = preprocess_inputs(data) 

In [ ]:
# Training the data

model = MLPClassifier()
model.fit(X_train, y_train)

In [ ]:
print(f"Neural network model accuracy:{model.score(X_test, y_test)*100}") 

In [ ]:
y_preds = model.predict(X_test)

In [ ]:
# Checking the number of correct predictions

np.sum((y_preds==y_test))

In [ ]:
# Total number of values in test data

y_test.shape

In [ ]:
# Checking accuracy manually

np.sum((y_preds==y_test))/y_test.shape